In [3]:
pip install cvxopt

     |████████████████████████████████| 3.1 MB 2.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [34]:
import random

from math import sqrt
from numpy import array
import scipy.linalg as linalg

from cvxopt import matrix
from cvxopt.blas import dot
from cvxopt.solvers import qp, options

n = 4

nbMarkets = 3
capacity = 100.0


slope = [-8, -15, -6] 
WTP = [1000, 1020, 950]
cost = [8, 15, 950]

P = matrix(0.0, (nbMarkets+1,nbMarkets+1))  #+1 since we add a variable for the spare  P[i,j] is the i-th row, j-th column value
for i in range(nbMarkets) : P[i,i] = -2*slope[i]
    
q = matrix(0.0, (nbMarkets+1,1))
for i in range(nbMarkets) : q[i,0] = cost[i] - WTP[i]
    
#Build matrices : matrix(v, (n,m)) will build a matrix of n rows, m columns, containing the value "v" in each cell
G = matrix([[-1.0,0.0,0.0,0.0],
            [0.0,-1.0,0.0,0.0],
            [0.0,0.0,-1.0,0.0],
            [0.0,0.0,0.0,-1.0]])

h = matrix(0.0, (n,1))

A = matrix(1.0, (1,n))

b = matrix(capacity)
    
xOpt =qp(P, q, G, h, A, b)
print('Revenue = ', -xOpt['primal objective'])
print('Trade matrix : %.1f to market1, %.1f to market2, %.1f to market3, %.1f of spare'%(xOpt['x'][0],xOpt['x'][1],xOpt['x'][2],xOpt['x'][3])) 
type(A)

            
        


     pcost       dcost       gap    pres   dres
 0: -4.7492e+04 -4.8884e+04  1e+03  0e+00  8e-02
 1: -4.7581e+04 -4.7642e+04  6e+01  9e-16  1e-03
 2: -4.7585e+04 -4.7588e+04  2e+00  7e-15  1e-05
 3: -4.7586e+04 -4.7586e+04  1e-01  1e-14  2e-07
 4: -4.7586e+04 -4.7586e+04  1e-02  3e-14  7e-17
Optimal solution found.
Revenue =  47585.7438553853
Trade matrix : 62.0 to market1, 33.5 to market2, 0.0 to market3, 4.5 of spare


cvxopt.base.matrix

In [24]:
qp?

Signature:
qp(
    P,
    q,
    G=None,
    h=None,
    A=None,
    b=None,
    solver=None,
    kktsolver=None,
    initvals=None,
    **kwargs,
)
Docstring:
Solves a quadratic program

    minimize    (1/2)*x'*P*x + q'*x
    subject to  G*x <= h
                A*x = b.


Input arguments.

    P is a n x n dense or sparse 'd' matrix with the lower triangular
    part of P stored in the lower triangle.  Must be positive
    semidefinite.

    q is an n x 1 dense 'd' matrix.

    G is an m x n dense or sparse 'd' matrix.

    h is an m x 1 dense 'd' matrix.

    A is a p x n dense or sparse 'd' matrix.

    b is a p x 1 dense 'd' matrix or None.

    solver is None or 'mosek'.

    The default values for G, h, A and b are empty matrices with
    zero rows.


Output arguments (default solver).

    Returns a dictionary with keys 'status', 'x', 's', 'y', 'z',
    'primal objective', 'dual objective', 'gap', 'relative gap',
    'primal infeasibility, 'dual infeasibility', 'primal slack',

In [25]:
import numpy as np
from pulp import *


def myQuadraticFunction(x) : #x is an np.array of lenghth = 3
    quadraticTerm = - np.sum(np.array(slope)*x*x)
    linearTerm = np.dot(x, np.array(cost) - np.array(WTP))
    return quadraticTerm + linearTerm


prob = LpProblem("MarketProblem",LpMinimize)
#flows_Var = ...
#Z_Var = ... 
# prob += ... set objective function
# prob += ... set capacity constraint



#Generate a set of point x_k around a starting point x0 and put it in the dict such that hyperplanDict['xk'] is a np.array with the 3 components

step_size = 0.5
nbStep = 22
x0 = [30.0, 10.0, 40.0]

hyperplanDict = {}
counter=0
for i1 in range(nbStep):
    for i2 in range(nbStep):
        for i3 in range(nbStep):
            hyperplanDict['x%s'%(counter)] = np.array([x0[0] + step_size * i1, x0[1]  + step_size * i2, x0[2]  + step_size * i3])
            counter+=1
        
        

for i in hyperplanDict : 
    x = hyperplanDict[i]  #a np.array of lenght 3
    
    #hyperplanSlope = ... a np.array of length 3
    #intercept = ... a scalar

    #prob += ... add the linear solver


    
#Solve and get the solution
solution = [0,0,0]  #initiate the solution
prob.solve()
print("Status:", LpStatus[prob.status])
print ("Objective = ", value(prob.objective))
prob.writeLP("marketProblem.lp", writeSOS=1, mip=1)
for v in prob.variables(): 
    print(v.name, v.varValue)
    for i in range(3) : 
        if v.name == "Flows_%s"%(i) : solution[i] = v.varValue
            
print("f(%s)=%f"%(solution,myQuadraticFunction(np.array(solution))))


Status: Optimal


AttributeError: 'NoneType' object has no attribute 'value'

In [26]:
x = np.array([36.2, 20, 44])  
print(myQuadraticFunction(x))

-69472.48
